In [ ]:
# Instalar Java Tool Kit
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar Spark
!wget -q https://downloads.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

# Descompactar Spark
!tar xf /content/spark-3.4.1-bin-hadoop3.tgz

# Instalar lib findspark que ajuda a localizar o Spark no sistema e importa-lo como uma biblioteca regular
!pip install -q findspark

In [ ]:
# Lib os - Permite Interagir com S.O - Cria variaveis de ambiente necessarias
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

In [ ]:
# Inicializa Findspark - Facilita Uso de Spark em Notebook
import findspark
findspark.init()

In [ ]:
# Start SparkContext e StreamingContext
from pyspark.context import SparkContext
from pyspark.streaming import StreamingContext
from time import sleep

sc = SparkContext(appName="DStream Example")
ssc = StreamingContext(sc, 2)  # Define Leitura dos Dados a Cada 2 Segundos

In [ ]:
# Gera 3 Grupos, Listas de 1 a 10
rddQueue = []
for i in range(3):
    rddQueue += [ssc.sparkContext.parallelize([j for j in range(1, 11)])] # Numero de 1 a 10

In [ ]:
# Gera Objeto DStream com Listas Geradas
inputStream = ssc.queueStream(rddQueue)

# Confere Tipo do Objeto Gerado
type(inputStream)

pyspark.streaming.dstream.DStream

In [ ]:
# Executa Transformações map / reduceByKey Sobre Dados Lidos
mappedStream = inputStream.map(lambda x: (x % 2, 1)) # Gera Chave/Valor -> Chave = Resto Div 2
reducedStream = mappedStream.reduceByKey(lambda a, b: a + b) # Soma Valores por Chave

In [ ]:
# Executa Ação pprint Sobre Dados
reducedStream.pprint()

In [ ]:
# Inicia Processamento Streaming
ssc.start()

# Aguarda 6 Segundos antes do Stop
sleep(6)

# stopSparkContext -> True para Contexto Spark alem do Contexto Streaming
# stopGraceFully   -> True aguarda processamento pendente concluir antes de interromper
ssc.stop(stopSparkContext=True, stopGraceFully=True)

-------------------------------------------
Time: 2023-11-03 18:50:48
-------------------------------------------
(0, 5)
(1, 5)

-------------------------------------------
Time: 2023-11-03 18:50:50
-------------------------------------------
(0, 5)
(1, 5)

-------------------------------------------
Time: 2023-11-03 18:50:52
-------------------------------------------
(0, 5)
(1, 5)

-------------------------------------------
Time: 2023-11-03 18:50:54
-------------------------------------------

-------------------------------------------
Time: 2023-11-03 18:50:56
-------------------------------------------



In [ ]:
# Parar SparkContext - Somente Executar se Precisar Reprocessar o Streaming
ssc.stop(stopSparkContext=True, stopGraceFully=True)